# Script Kullanımı Açıklama:
Script'imiz bir Networkx graph'ı alıyor ve bu graph'ı 30 dimension'lı bir graph embedding'ine çeviriyor
Bu graph embedding'ini Node2Vec kütüphanesini kullanarak elde etmekteyiz 
Parameters:
1) walk_length = 10 
2) num_walks = 40

pickle kütüphanesini kullanarak elde ettiğimiz eğitilmiş kMeans için gerekli olan modelleri load ediyoruz.
Load Ettiğimiz modellerle bize verilen graph'ın sonucunu predict ederek bu predict edilen sonucu dönüyoruz.

# Kullanma yöntemi:
Networkx graph'ınızı "scriptKullanilacakFonksiyon" isimli fonksiyona vermeniz yeterlidir.
Bu fonksiyon graph'ınızı alacak ve sonucunda prediction'ı dönecektir.
Fonksiyonda dönen değer 1 ise Partition edilebilir (Yani Yes label'lı), 0 ise Partition edilemez(Yani No label'li)

# Gerekli kutuphaneler
1) numpy
2) node2Vec (Version 0.4.6)
3) pickle   (Version 0.7.5)

In [33]:
# General Libraries
import random as rand
import numpy as np
# Graph Representation & Embedding Library
import networkx as nx 
from node2vec import Node2Vec
import pickle

import differentGraphs as dg

In [34]:
def getEmbedding(G):
    # Precompute probabilities and generate walks
    node2vec = Node2Vec(G, dimensions=30, walk_length=10, num_walks=40, workers=4)

    # Embed nodes
    model = node2vec.fit(window=5, min_count=1, batch_words=4)

    # Get embeddings for all nodes in the graph
    all_node_embeddings = {node: model.wv[str(node)] for node in G.nodes()}

    return all_node_embeddings
    # loaded_model = Node2Vec.load("node2vec_model.bin")
    # loaded_embedding = loaded_model.wv['0']
    # print(f"Loaded Embedding for Node 0: {loaded_embedding}")

def dictionaryToNpArray(embedding):
    array_from_dict = np.array(list(embedding.values()))
    return array_from_dict


In [35]:
def scriptKullanilacakFonksiyon(G):
    # Modeli yükleyip kullanma
    # K-Means modelini yükle
    with open('kmeans_model.pickle', 'rb') as kmeans_file:
        loaded_kmeans = pickle.load(kmeans_file)

    # Lineer Regresyon modellerini yükle
    loaded_models = []
    for i in range(2):
        with open(f'linear_regression_model_cluster_{i}.pickle', 'rb') as model_file:
            loaded_model = pickle.load(model_file)
        loaded_models.append(loaded_model)

    k_neighbors = rand.randint(2,4)
    probability = rand.random()
    numberOfNodes = rand.randint(60,200)
    G = dg.generate_watts_strogatz_graph(numberOfNodes,seed = 123, k_neighbors=k_neighbors, probability= probability)

    while not nx.is_connected(G):
        k_neighbors = rand.randint(2,4)
        probability = rand.random()
        numberOfNodes = rand.randint(60,200)
        G = dg.generate_watts_strogatz_graph(numberOfNodes,seed = 123, k_neighbors=k_neighbors, probability= probability)
    
    nodeEmbeddings = getEmbedding(G) 
    nodeEmbeddingsArray = dictionaryToNpArray(nodeEmbeddings)
    graphEmbedding = np.mean(nodeEmbeddingsArray, axis=0).astype('float')

    cluster_prediction = loaded_kmeans.predict(graphEmbedding.reshape(1, -1))

    # Tahmin edilen kümenin modelini kullanarak sonucu tahmin et
    result_prediction = loaded_models[cluster_prediction[0]].predict(graphEmbedding.reshape(1, -1))
    
    # Prediction 0.5'den buyukse 1 veriyor (Yani partition edilebilir)
    if result_prediction > 0.5:
        return 1
    
    # Prediction 0.5'den kucukse 0 veriyor (Yani partition edilemedi)
    else: 
        return 0

In [36]:
print(scriptKullanilacakFonksiyon(1))

c:\Users\bora\anaconda3\envs\test1\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator KMeans from version 1.3.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\Users\bora\anaconda3\envs\test1\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LinearRegression from version 1.3.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Computing transition probabilities: 100%|██████████| 135/135 [00:00<00:00, 26991.66it/s]


0
